In [54]:
from nltk.tokenize import word_tokenize
import pandas as pd
import spacy 
import matplotlib.pyplot as plt
import gzip,os,glob
import numpy as np
import pickle
import nltk
import codecs
from langdetect import detect
import csv
from tqdm import tqdm

In [55]:
# load data from pkl file 
df = pd.read_pickle('../data/data_filtered contents.pkl')
print(df.shape)

(20247, 5)


In [56]:
# Adds new collumn, detects language of column
df['Languagereview'] = df['content'].apply(detect)

In [57]:
# delets all rows with the value "en"
df = df[df.Languagereview == "de"]

In [58]:
# resets index from 0, 4, 11, ... (random) to 0,1,2, ...
df.reset_index(inplace=True,drop=True)
print(df.shape)

(18142, 6)


In [59]:
df_movies = pd.read_pickle('../data/df_movies_cleaning_2.pkl')
df_movies

,title1,title2,title3,content,word_count
0,Electric gir,None,ein Film von Ziska Riemann Victoria Schulz ...,Eine frischgebackene Synchronsprecherin identi...,88
1,Hans Hartung - malen so schnell wie der Blitz,Hans Hartung - la fureur de peindre.,None,Er konnte malen so schnell wie der Blitz: Der ...,103
2,Der Fotograf Roger Melis - Chronist der Ostd...,None,Buch und Regie: Pamela Meyer-Arndt.,In seinen Reportagen zeigte Roger Melis den Al...,71
3,Ayk,None,Regie: Sergey Dvortsevoy Drehbuch: Sergey Dv...,Eine illegal unter erbärmlichen Umständen in M...,88
4,The unanswered Ives,Wunderkind Wall-Street-Gigant Klangpionier,ein Film von Anne-Katrin Peitz.,"8 # a ""Charles Ives (1874-1954) ist ein Man...",109
...,...,...,...,...,...
7443,Jochen Roller Let's dance! Lecture performance,None,Konzept und Leitung: Gabriele Brandstetter.,Jochen Roller geboren 1971 in Berlin studier...,90
7444,Ai Weiwei - Stimmen gegen das Verstummen,None,ein Film von Birgit Adler-Conrad.,Der chinesische Künstler Ai Weiwei wurde im Fr...,141
7445,New York tomorrow,None,réalisation: Yoann Le Gruiec. Scénario: Jean-M...,"Die Webdokumentation ""New York 3.0"" gibt Inter...",55
7446,Hostess,None,ein Film von Rolf Römer. Mit: Annekathrin Bürg...,Eine Hostess die seit zwei Jahren mit einem A...,58


In [60]:
# Removing non-alphabetical symbols in dataframe, keeping umlauts
df_movies['content'] = df_movies['content'].replace('[^a-zA-Z \äöüß]', '', regex=True).str.replace(' +',' ').str.strip()
# Transform umlauts
# df_movies['content'] = df_movies['content'].str.replace(chr(252), 'ue')
# df_movies['content'] = df_movies['content'].str.replace(chr(228), 'ae')
# df_movies['content'] = df_movies['content'].str.replace(chr(246), 'oe')
# df_movies['content'] = df_movies['content'].str.replace(chr(196), 'Ae')
# df_movies['content'] = df_movies['content'].str.replace(chr(214), 'Oe')
# df_movies['content'] = df_movies['content'].str.replace(chr(220), 'Ue')
# df_movies['content'] = df_movies['content'].str.replace(chr(223), 'ss')
# Lowercase
df_movies['content'] = df_movies['content'].str.lower()
# rename collumn
df_movies.rename(columns={'title1': 'title'}, inplace=True)
df_movies.rename(columns={'title3': 'author'}, inplace=True)
# drop collumn
df_movies.drop(['word_count'],1,inplace=True)
df_movies.drop(['title2'],1,inplace=True)
# add label collumn
df_movies['medium'] ='movie'
# add nan
df_movies.fillna(value=pd.np.nan, inplace=True)
df_movies.head()

C:\Users\Johannes\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


,title,author,content,medium
0,Electric gir,ein Film von Ziska Riemann Victoria Schulz ...,eine frischgebackene synchronsprecherin identi...,movie
1,Hans Hartung - malen so schnell wie der Blitz,NaN,er konnte malen so schnell wie der blitz der k...,movie
2,Der Fotograf Roger Melis - Chronist der Ostd...,Buch und Regie: Pamela Meyer-Arndt.,in seinen reportagen zeigte roger melis den al...,movie
3,Ayk,Regie: Sergey Dvortsevoy Drehbuch: Sergey Dv...,eine illegal unter erbärmlichen umständen in m...,movie
4,The unanswered Ives,ein Film von Anne-Katrin Peitz.,a charles ives ist ein mann der dissonanzen un...,movie


In [61]:
# drop nan in ['content']
df_movies = df_movies[df_movies['content'].notna()]

In [62]:
# Removing non-alphabetical symbols in dataframe, keeping umlauts
df['content'] = df['content'].replace('[^a-zA-Z \äöüß]', '', regex=True).str.replace(' +',' ').str.strip()
# Transform umlauts
# df['content'] = df['content'].str.replace(chr(252), 'ue')
# df['content'] = df['content'].str.replace(chr(228), 'ae')
# df['content'] = df['content'].str.replace(chr(246), 'oe')
# df['content'] = df['content'].str.replace(chr(196), 'Ae')
# df['content'] = df['content'].str.replace(chr(214), 'Oe')
# df['content'] = df['content'].str.replace(chr(220), 'Ue')
# df['content'] = df['content'].str.replace(chr(223), 'ss')
# Lowercase
df['content'] = df['content'].str.lower()
# add label collumn
df['medium'] ='print'
df.head()

,author,title,class,content,word_count,Languagereview,medium
0,NaN,Aernout Mik - Communitas,[Kunstgeschichte],aernout mik zählt zu den wichtigsten künstlern...,64,de,print
1,NaN,Anton Henning : Dt. /Engl.,NaN,die vorliegende publikation widmet sich drei o...,113,de,print
2,"Specht, Heike","Die Feuchtwangers : Familie, Tradition und j...","[Sprachwissenschaften, Geschichte]",jahre geschichte deutschjüdischen bürgertums a...,169,de,print
3,"Scruton, Roger",Grüne Philosophie : Ein konservativer Denkan...,"[Allgemeines, Philosophie, Theologie und Relig...",der umweltschutz liegt herkömmlich im hoheitsg...,101,de,print
4,"Schlagowski, Heinz",Technische Dokumentation im Maschinen- und An...,[Technik],das buch gibt dem anwender die entscheidenden ...,156,de,print


In [63]:
# drop nan in ['content']
df = df[df['content'].notna()]

<h1>Combining content</h1>

In [64]:
df['title'] = df['title'].str.split('  / von').str[0]
df['title'] = df['title'].str.replace('[^\w\s]','')
df['title'] = df['title'].replace('\s+', ' ', regex=True)
df['title'] = df['title'].str.lower()

In [65]:
df['author'] = df['author'].str.split('  / von').str[0]
df['author'] = df['author'].str.replace('[^\w\s]','')
df['author'] = df['author'].replace('\s+', ' ', regex=True)
df['author'] = df['author'].str.lower()

In [66]:
df_combined = df.copy()
df_combined.drop(['class','word_count', 'Languagereview'],axis=1, inplace=True)
df_combined.replace(np.nan, '', regex=True, inplace=True)
df_combined.replace('\s+', ' ', regex=True, inplace=True)

df_combined['content_nltk'] = df_combined['title'] +" "+ df_combined['content']
df_combined['content_nltk'] = df_combined['content_nltk'].str.strip()

df_movies['title'] = df_movies['title'].str.lower()

df_movies['content_nltk'] = df_movies['title'] +" "+ df_movies['content']
df_movies['content_nltk'] = df_movies['content_nltk'].str.strip()

# df['content_nltk'] = df_combined
# df_movies['content_nltk'] = df_movies_combined

# whitespaces to nan
df_combined = df_combined.replace(r'^\s*$', np.nan, regex=True)

In [67]:
df_combined.replace('\s+', ' ', regex=True, inplace=True)
df_combined = pd.concat([df_combined, df_movies])
df_combined.reset_index(inplace=True,drop=True)
df_combined = df_combined[['title','author','content_nltk','medium']]
df_combined

,title,author,content_nltk,medium
0,aernout mik communitas,NaN,aernout mik communitas aernout mik zählt zu de...,print
1,anton henning dt engl,NaN,anton henning dt engl die vorliegende publikat...,print
2,die feuchtwangers familie tradition und jüdis...,specht heike,die feuchtwangers familie tradition und jüdisc...,print
3,grüne philosophie ein konservativer denkansatz,scruton roger,grüne philosophie ein konservativer denkansatz...,print
4,technische dokumentation im maschinen und anl...,schlagowski heinz,technische dokumentation im maschinen und anla...,print
...,...,...,...,...
25585,jochen roller let's dance! lecture performance,Konzept und Leitung: Gabriele Brandstetter.,jochen roller let's dance! lecture performan...,movie
25586,ai weiwei - stimmen gegen das verstummen,ein Film von Birgit Adler-Conrad.,ai weiwei - stimmen gegen das verstummen der c...,movie
25587,new york tomorrow,réalisation: Yoann Le Gruiec. Scénario: Jean-M...,new york tomorrow die webdokumentation new yor...,movie
25588,hostess,ein Film von Rolf Römer. Mit: Annekathrin Bürg...,hostess eine hostess die seit zwei jahren mit ...,movie


<h1>Remove Stop Words</h1>

In [68]:
from nltk.corpus import stopwords
stop = stopwords.words('german')

df_combined['content_nltk'] = df_combined['content_nltk'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_combined['content_nltk'].iloc[3]

'grüne philosophie konservativer denkansatz umweltschutz liegt herkömmlich hoheitsgebiet politischen linken bedrohung natur zufolge internationalen kapitalismus konsumverhalten unbegrenzten ausbeutung natürlicher ressourcen zuzuschreiben roger scruton entblößt grundverständnis gefährlichen trugschluss zerstörerisch kosysteme wirkt denen zukunft abhängt konservative denker würdigt traditionelle prinzipien offensichtlichste wirksamste mittel erhalt planeten sichern lokalismus bürgerverantwortung schlagen dabei bemühungen schwerfälliger nichtregierungsorganisationen internationaler komitees zukunft mitnichten gesichert roger scruton beweist gedankenreichen schrift fortbestand kosystems erde möglich'

<h1>Tokenizing</h1>

In [69]:
df_tok = df_combined['content_nltk']
df_tok

0        aernout mik communitas aernout mik zählt wicht...
1        anton henning dt engl vorliegende publikation ...
2        feuchtwangers familie tradition jüdisches selb...
3        grüne philosophie konservativer denkansatz umw...
4        technische dokumentation maschinen anlagenbau ...
                               ...                        
25585    jochen roller let's dance! lecture performance...
25586    ai weiwei - stimmen verstummen chinesische kün...
25587    new york tomorrow webdokumentation new york gi...
25588    hostess hostess seit zwei jahren automechanike...
25589    bruno peinado bruno peinado plastische kunst r...
Name: content_nltk, Length: 25590, dtype: object

In [70]:
tokenized = [ nltk.word_tokenize( str(sentence), language='german' ) for sentence in df_tok ]

In [71]:
print (len(tokenized))

25590


<h1>Lemmetization</h1>

In [72]:
from HanTa import HanoverTagger as ht

tagger = ht.HanoverTagger('morphmodel_ger.pgz')

In [73]:
words_lematized = []
for inner_list in tqdm(tokenized):
  temp = []
  for word in inner_list:
    temp.append(tagger.analyze(word))
  words_lematized.append(temp)    

100%|████████████████████████████████████████████████████████████████████████████| 25590/25590 [38:00<00:00, 11.22it/s]


In [74]:
words_lematized[666]

[('radikal', 'ADJA'),
 ('Innovation', 'NN'),
 ('Mobilität', 'NN'),
 ('technisch', 'ADJA'),
 ('betriebswirtschaftlich', 'ADJA'),
 ('Aspekt', 'NN'),
 ('radikal', 'ADJA'),
 ('Innovation', 'NN'),
 ('insbesondere', 'ADV'),
 ('Bergang', 'NN'),
 ('Elektromobilität', 'NN'),
 ('stellen', 'VVINF'),
 ('Wissenschaft', 'NN'),
 ('Wirtschaft', 'NN'),
 ('groß', 'ADJA'),
 ('Herausforderung', 'NN'),
 ('Bedarf', 'NN'),
 ('konsequent', 'ADJA'),
 ('koordiniert', 'ADJA'),
 ('Anstrengung', 'NN'),
 ('Schnittstelle', 'NN'),
 ('betriebswirtschaftlich', 'ADJA'),
 ('Ingenieurwissenschaftlichen', 'NN'),
 ('Forschung', 'NN'),
 ('umsetzen', 'VVIZU'),
 ('Juni', 'NN'),
 ('werden', 'VAFIN'),
 ('Duisburg', 'NE'),
 ('darüber', 'PROAV'),
 ('diskutieren', 'VVFIN'),
 ('Innovation', 'NN'),
 ('erforderlich', 'ADJD'),
 ('steueren', 'VVPP'),
 ('Mobilitätsstrategie', 'NN'),
 ('erwartet', 'ADJD'),
 ('Tagungsband', 'NN'),
 ('präsentieren', 'VVFIN'),
 ('Beitrag', 'NN'),
 ('Wissenschaftsforum', 'NN'),
 ('Mobilität', 'NN'),
 ('Univer

In [75]:
with open('../data/after_nltk/data_movies_content_after_nltk.pkl', 'wb') as f:
    pickle.dump(words_lematized, f)

In [90]:
with open('../data/after_nltk/data_movies_content_after_nltk.pkl', 'rb') as f:
    words_lematized_imp = pickle.load(f)
print(len(words_lematized_imp))

25590


In [91]:
# filterning the words according to their POS type

# words_lematized_filt = []

# for i in words_lematized:
#     temp = [j[0] for j in i if j[1].startswith(("NE", "NN", "VVFIN", "ADJD"))]
#     words_lematized_filt.append(temp)
    
# POS tagger list
# https://medium.com/@muddaprince456/categorizing-and-pos-tagging-with-nltk-python-28f2bc9312c3


In [92]:
# droping POS tags

words_lematized_2 = []
for inner_list in words_lematized_imp:
  temp = []
  for word in inner_list:
    temp.append(word[0])
  words_lematized_2.append(temp)

In [93]:
print (len(words_lematized_2))

25590


In [94]:
# # ä
# words_lematized_3 = []
# for inner_list in words_lematized_2:
#   temp = []
#   for word in inner_list:
#     temp.append(word.replace('ä', 'ae'))
#   words_lematized_3.append(temp)

In [95]:
# # Transforming Umlauts
# words_lematized_3 = []
# for inner_list in words_lematized_2:
#   temp = []
#   for word in inner_list:
#     temp.append(word.replace('ä', 'ae').replace(chr(246), 'oe').replace(chr(252), 'ue').replace(chr(223), 'ss').replace(chr(220), 'Ue').replace(chr(196), 'Ae').replace(chr(214), 'Oe').lower())
#   words_lematized_3.append(temp)

In [96]:
# lower case
words_lematized_3 = []
for inner_list in words_lematized_2:
  temp = []
  for word in inner_list:
    temp.append(word.lower())
  words_lematized_3.append(temp)

In [102]:
df_combined['content_nltk'] = pd.Series(words_lematized_3)

In [103]:
print(df_combined['content_nltk'].isnull().sum().sum())
print(df_combined.shape)

0
(25590, 4)


In [104]:
df_combined.head(20)

,title,author,content_nltk,medium
0,aernout mik communitas,NaN,"[aernout, mik, communitas, aernout, mik, zähle...",print
1,anton henning dt engl,NaN,"[anton, henning, dt, engl, vorliegend, publika...",print
2,die feuchtwangers familie tradition und jüdis...,specht heike,"[feuchtwanger, familie, tradition, jüdisch, se...",print
3,grüne philosophie ein konservativer denkansatz,scruton roger,"[grüne, philosophie, konservativ, denkansatz, ...",print
4,technische dokumentation im maschinen und anl...,schlagowski heinz,"[technisch, dokumentation, maschine, anlagenba...",print
5,geotechnik bodenmechanik,möller gerd,"[geotechnik, bodenmechanik, buch, vermitteln, ...",print
6,farben natur technik kunst,welsch norbert,"[farbe, natur, technik, kunst, farbe, auflage,...",print
7,geotechnik grundbau,möller gerd,"[geotechnik, grundbau, komplex, technisch, hoc...",print
8,einführung in die ölhydraulik ehem unter reni...,matthies hans jürgen,"[einführung, ölhydraulik, eher, renius, karl, ...",print
9,trust but test das vertrauen in virtuellen ge...,thiedeke udo,"[trust, but, test, vertrauen, virtuell, gemein...",print


<h1>Export</h1>

In [105]:
# export df
df_combined.to_pickle("../data/after_nltk/data_combined_after_nltk.pkl")

In [ ]:
# export for doc2vec
words_lematized_export = pd.Series(words_lematized_3)
words_lematized_export.to_csv('../data/after_nltk/data_movies_content_after_nltk_doc2vec.csv', index=True)

In [ ]:
# export for RRI
words_lematized_export_2 = words_lematized_export.str.join(' ')
words_lematized_export_2 = words_lematized_export_2.str.replace('[^\w\s]','')
words_lematized_export_2 = words_lematized_export_2.str.findall('\w{2,}').str.join(' ')
words_lematized_export_2.to_csv('../data/after_nltk/data_movies_content_after_nltk_rri.csv', index=True)

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [ ]:
df_import = pd.read_pickle('../data/data_filtered contents.pkl')

#df_combined['content'] = 

In [ ]:
df_combined.to_pickle('../data/after_nltk/data_movies_after_nltk.pkl')
# df_content.to_csv('../data/after_nltk/df_content.csv', index=True)
#df["content_nltk"].to_csv('../data/after_nltk/df_content_after_nltk+titles.csv', index=True)
#df["content_nltk"].to_csv('../data/after_nltk/df_movies_content_after_nltk.csv', index=True)
# df.to_excel('../data/after_nltk/data_after_nltk.xlsx', index = False)

In [ ]:
# #counts number of lines
# fname = "df_content.txt"
# count = 0
# with open(fname, 'r') as f:
#     for line in f:
#         count += 1
# print("Total number of lines is:", count)